# Multi Agent 2


### Patterns 

   - Reflection : 자기 비판
   - Reflexion : 현재 답변의 약점 탐색

##### Pattern 1 : Reflection

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from typing import List, Sequence
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph
import boto3
import botocore
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser
import json
import os
import sys

In [ ]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 블로그를 평가하는 인플루언서입니다. 사용자의 글에 대한 비평과 추천을 생성하세요."
            "길이, 바이럴율, 스타일 등에 대한 요청을 포함하여 자세한 추천을 제공하세요.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 게시물을 작성하는 업무를 맡은 기술 인플루언서의 보조자입니다."
            " 사용자의 요청에 따라 최상의 게시물을 작성하세요."
            " 사용자가 비평을 제공하면, 이전 시도를 수정하여 250개 단어로 답변하세요.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# 1. 모델 정의
# model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))
model_id = "amazon.nova-lite-v1:0"
model_kwargs =  { 
        "temperature": 0,
        "top_p": 1,
}

llm = ChatBedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

generate_chain = generation_prompt | llm
reflect_chain = reflection_prompt | llm

In [ ]:
REFLECT = "reflect"
GENERATE = "generate"


def generation_node(state: Sequence[BaseMessage]):
    return generate_chain.invoke({"messages": state})


def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    res = reflect_chain.invoke({"messages": messages})
    return [HumanMessage(content=res.content)]


builder = MessageGraph()
builder.add_node(GENERATE, generation_node)
builder.add_node(REFLECT, reflection_node)
builder.set_entry_point(GENERATE)


def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        return END
    return REFLECT


builder.add_conditional_edges(GENERATE, should_continue)
builder.add_edge(REFLECT, GENERATE)

graph = builder.compile()


In [ ]:
inputs = HumanMessage(content="""다음의 블로그 내용을 더 멋지게 만들어 주세요:"

            — Agentic AI는 최근 LLM 기반 시스템의 진화된 형태로, **"자율적이고 목표 지향적인 행동을 할 수 있는 AI"**를 말합니다. 

                                  """)
response = graph.invoke(inputs)

print(response)

##### Pattern 2 : Reflexion